### Create GeoJSON files for the following:
- 1x1 degree cells for Texas used to call Baron point query region APIs
- points returned by Baron point query region APIs

In [1]:
from shapely.geometry import Polygon
import leafmap
import geopandas as gpd
import pandas as pd
from dotenv import load_dotenv
import os
import time
import base64
import datetime
import hmac
from hashlib import sha1 as sha
import json
import urllib.request
import requests

In [2]:
load_dotenv("local.env")


host = "http://api.velocityweather.com/v1"
access_key = os.getenv("BARON_KEY")
if access_key is None:
    raise ValueError("Missing BARON_KEY in local.env file")
access_key_secret = os.getenv("BARON_SECRET")
if access_key_secret is None:
    raise ValueError("Missing BARON_SECRET in local.env file")


def sign_old(string_to_sign, secret):
    """ Returns the signature for string_to_sign
    """
    signature_bytes = base64.urlsafe_b64encode(
        hmac.new(secret.encode('utf-8'), string_to_sign.encode('utf-8'), sha).digest())
    signature = signature_bytes.decode('utf-8')

    return signature.replace('/', '_').replace('+', '-').rstrip('=')


def sign(string_to_sign, secret):
    hmac_sha1 = hmac.new(
        # Convert secret to bytes using UTF-8
        secret.encode('utf-8'),
        # Convert input string to bytes using UTF-8
        string_to_sign.encode('utf-8'),
        sha                        # Use SHA1 hash algorithm
    )

    # Get the binary digest
    hmac_digest = hmac_sha1.digest()

    # Encode to base64
    base64_encoded = base64.b64encode(hmac_digest).decode('utf-8')

    # Replace characters to make URL safe
    signature = base64_encoded.replace('/', '_').replace('+', '-')

    return signature


def sign_request(url, key, secret):
    """ Returns signed url
    """

    ts = str(int(time.time()))
    sig = sign(key + ":" + ts, secret)
    q = '?' if url.find("?") == -1 else '&'
    url += "%ssig=%s&ts=%s" % (q, sig, ts)
    return url

In [3]:
def point_query_region(product, product_config, region_bounds):
    meta_url = f'{host}/{access_key}/meta/tiles/product-instances/{product}/{product_config}?page_size=1'
    meta_url = sign_request(meta_url, access_key, access_key_secret)

    try:
        response = requests.get(meta_url)
    except requests.exceptions.RequestException as e:
        print('Request failed:', e)
        return
    assert response.status_code == 200

    # Decode the product instance response and get the most recent product instance time,
    # to be used in the WMS image request.
    content = json.loads(response.text)
    product_instance_time = content[0]['time']

    w_lon = region_bounds[0]
    n_lat = region_bounds[1]
    e_lon = region_bounds[2]
    s_lat = region_bounds[3]
    query_region = f'w_lon={w_lon}&n_lat={n_lat}&e_lon={e_lon}&s_lat={s_lat}'

    url = f'{host}/{access_key}/point/region/{product}/{product_config}/{product_instance_time}.json?{query_region}'
    url = sign_request(url, access_key, access_key_secret)

    headers = {
        'Accept': 'application/json',
        'Accept-Encoding': 'gzip'
    }

    try:
        response = requests.get(url, headers=headers)
        print(f'Response status code: {response.status_code}')
        # print(f'Response headers: {response.headers}')
        # print(f'Response content: {response.content}')
    except requests.exceptions.RequestException as e:
        print('Request failed:', e)
        return
    assert response.status_code == 200

    content = json.loads(response.text)

    return content

In [4]:
hc_lat = 34.5
hc_long = -101.5
m = leafmap.Map(center=[hc_lat, hc_long], zoom=7,
                epsg="4326")

In [5]:
tx_panhandle_bound_box = [-103.05, 34.65, -99.99, 36.53]
tx_panhandle_bound_box_polygon = Polygon([
    (tx_panhandle_bound_box[0], tx_panhandle_bound_box[1]),
    (tx_panhandle_bound_box[2], tx_panhandle_bound_box[1]),
    (tx_panhandle_bound_box[2], tx_panhandle_bound_box[3]),
    (tx_panhandle_bound_box[0], tx_panhandle_bound_box[3]),
])

# Create a GeoDataFrame from the bounding box polygon
pan_gdf = gpd.GeoDataFrame(
    {'geometry': [tx_panhandle_bound_box_polygon]},
    crs="EPSG:4326"
)

In [6]:
def frange(start, stop, step):
    """Generate a range of floating point numbers."""
    while start < stop:
        yield round(start, 6)
        start += step

In [7]:
def process_response(response):
    """Process the response and return a DataFrame."""
    data = []
    for item in response:
        if item:
            # print(f"item={item}")
            for item_point in item:
                # print(f"item_point={item_point}")

                if 'value' in item_point and item_point['value'] > 30:
                        # use latitude and longitude from the item to create a point
                        
                        # add point to the GeoDataFrame

                    data.append({
                        'latitude': item_point["coordinates"][1],
                        'longitude': item_point["coordinates"][0],
                        'value': item_point['value']
                    })
    return pd.DataFrame(data)

In [8]:
texas_gdf = gpd.read_file("texas.geojson")
texas_gdf

,name,abbreviation,capital,city,population,area,waterarea,landarea,houseseats,statehood,group,geometry
0,Texas,TX,Austin,Houston,26448193,695660,19075,676588,36,1845-12-29,US States,"MULTIPOLYGON (((-96.83003 28.11184, -96.82705 ..."


In [9]:
# Texas Panhandle area
# min_lat, max_lat = 32.65, 36.66
# min_lon, max_lon = -103.05, -99.99

# bounding box for the entire state of Texas
min_lat, max_lat = 25.8, 36.8
min_lon, max_lon = -106.6, -93.5
lat_step = lon_step = 1.0

cells_gdf = None
points_gdf = None

for lat in frange(min_lat, max_lat - lat_step + 0.01, lat_step):
    for lon in frange(min_lon, max_lon - lon_step + 0.01, lon_step):
        print(f"Box: N:{lat+1}, S:{lat}, E:{lon+1}, W:{lon}")
        temp_bound_box = [lon, lat + lat_step, lon + lon_step, lat ]

        cell_polygon = Polygon([
            (lon, lat),
            (lon + lon_step, lat),
            (lon + lon_step, lat + lat_step),
            (lon, lat + lat_step),
        ])

        # Check if the cell polygon is within the Texas polygon
        if not cell_polygon.intersects(texas_gdf.geometry[0]):
            print(f"Box: N:{lat+1}, S:{lat}, E:{lon+1}, W:{lon} is outside of Texas")
            continue

        # add the polygon to the GeoDataFrame
        if cells_gdf is None:
            cells_gdf = gpd.GeoDataFrame({'geometry': [cell_polygon]}, crs="EPSG:4326")
        else:
            # Append the new row to the existing GeoDataFrame
            new_cell_gdf = gpd.GeoDataFrame({'geometry': [cell_polygon]}, crs="EPSG:4326")
            cells_gdf = pd.concat([cells_gdf, new_cell_gdf], ignore_index=True)

        # call Baron API
        response = point_query_region('C39-0x04A6-0', 'Standard-Geodetic', temp_bound_box)
        # print(f"Response: {response}")
        # print("---------------------")

        if response is None:
            print("No response")
            continue
        
        points_df = process_response(response)
        # print(f"Points: {points_df.head()}")

        if not points_df.empty:
            print(f"Points: {points_df.shape}")

            if points_gdf is None:
                # Create a GeoDataFrame from the points DataFrame
                points_gdf = gpd.GeoDataFrame(
                    points_df,
                    geometry=gpd.points_from_xy(points_df.longitude, points_df.latitude),
                    crs="EPSG:4326"
                )
            else:
                # Append the new points to the existing GeoDataFrame
                new_points_gdf = gpd.GeoDataFrame(
                    points_df,
                    geometry=gpd.points_from_xy(points_df.longitude, points_df.latitude),
                    crs="EPSG:4326"
                )
                points_gdf = pd.concat([points_gdf, new_points_gdf], ignore_index=True)
        
            print(f"Points GeoDataFrame: {points_gdf.shape}")

Box: N:26.8, S:25.8, E:-105.6, W:-106.6
Box: N:26.8, S:25.8, E:-105.6, W:-106.6 is outside of Texas
Box: N:26.8, S:25.8, E:-104.6, W:-105.6
Box: N:26.8, S:25.8, E:-104.6, W:-105.6 is outside of Texas
Box: N:26.8, S:25.8, E:-103.6, W:-104.6
Box: N:26.8, S:25.8, E:-103.6, W:-104.6 is outside of Texas
Box: N:26.8, S:25.8, E:-102.6, W:-103.6
Box: N:26.8, S:25.8, E:-102.6, W:-103.6 is outside of Texas
Box: N:26.8, S:25.8, E:-101.6, W:-102.6
Box: N:26.8, S:25.8, E:-101.6, W:-102.6 is outside of Texas
Box: N:26.8, S:25.8, E:-100.6, W:-101.6
Box: N:26.8, S:25.8, E:-100.6, W:-101.6 is outside of Texas
Box: N:26.8, S:25.8, E:-99.6, W:-100.6
Box: N:26.8, S:25.8, E:-99.6, W:-100.6 is outside of Texas
Box: N:26.8, S:25.8, E:-98.6, W:-99.6
Response status code: 200
Box: N:26.8, S:25.8, E:-97.6, W:-98.6
Response status code: 200
Points: (430, 3)
Points GeoDataFrame: (430, 4)
Box: N:26.8, S:25.8, E:-96.6, W:-97.6
Response status code: 200
Points: (1179, 3)
Points GeoDataFrame: (1609, 4)
Box: N:26.8, S

In [10]:
cells_gdf.to_file("tx_cells.geojson", driver="GeoJSON")

In [11]:
points_gdf = points_gdf.drop_duplicates(subset=['latitude', 'longitude'])
# Export the points GeoDataFrame to a GeoJSON file
points_gdf.to_file("tx_points.geojson", driver="GeoJSON")